In [11]:
# import libraries
import numpy as np
np.set_printoptions(precision = 4, suppress = True)
import pandas as pd
import os
from datetime import datetime
%matplotlib qt

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# import modules
from widgets import CandlestickWidget

In [12]:
best_series_dir = "./data/out/best-series"
in_data_dir = "./data/in"
n_levels = 4

In [13]:
def to_series(series, time_col="time"):
    series[time_col] = series[time_col].apply(lambda x: datetime.utcfromtimestamp(x))
    series.set_index(time_col, inplace=True)
    return series

In [14]:
candle_series = to_series(pd.read_csv(os.path.join(in_data_dir, "ohlcv-eth-usdt-1-min.csv"),
                                      sep='|', usecols=list(range(5)), header=None,
                                      names=["time", "open", "high", "low", "close"]))
open_order_series = to_series(pd.read_csv(os.path.join(best_series_dir, "open-order-series.csv")))
close_order_series = to_series(pd.read_csv(os.path.join(best_series_dir, "close-order-series.csv")))
entry_indic_series = to_series(pd.read_csv(os.path.join(best_series_dir, "entry-indic-series.csv")))
exit_indic_series = to_series(pd.read_csv(os.path.join(best_series_dir, "exit-indic-series.csv")))

In [15]:
entry_indic_series

,level 1,level 2,level 3,level 4
time,,,,
2018-01-05 10:34:00,823.159607,740.843689,658.527710,576.211731
2018-01-05 11:04:00,823.255005,740.929565,658.604065,576.278503
2018-01-05 11:34:00,823.273865,740.946472,658.619080,576.291687
2018-01-05 12:04:00,823.604553,741.244080,658.883667,576.523193
2018-01-05 12:34:00,823.971069,741.573975,659.176880,576.779724
...,...,...,...,...
2022-10-28 10:05:00,1264.416626,1137.974976,1011.533325,885.091614
2022-10-28 10:35:00,1263.879150,1137.491333,1011.103394,884.715454
2022-10-28 11:05:00,1263.149536,1136.834595,1010.519653,884.204651


In [16]:
close_order_series = close_order_series.reindex(candle_series.index)
open_order_series = open_order_series.reindex(candle_series.index)
entry_indic_series = entry_indic_series.reindex(candle_series.index)
exit_indic_series = exit_indic_series.reindex(candle_series.index)

In [17]:
entry_indic_series

,level 1,level 2,level 3,level 4
time,,,,
2017-08-17 04:00:00,NaN,NaN,NaN,NaN
2017-08-17 04:01:00,NaN,NaN,NaN,NaN
2017-08-17 04:02:00,NaN,NaN,NaN,NaN
2017-08-17 04:03:00,NaN,NaN,NaN,NaN
2017-08-17 04:04:00,NaN,NaN,NaN,NaN
...,...,...,...,...
2022-10-28 12:23:00,NaN,NaN,NaN,NaN
2022-10-28 12:24:00,NaN,NaN,NaN,NaN
2022-10-28 12:25:00,NaN,NaN,NaN,NaN


In [18]:
pair = "ETH/USDT"
interval = "1 min"
title = f"{pair}\n{interval}"

In [21]:
widget = CandlestickWidget(candle_series, entry_indic_series, exit_indic_series, open_order_series,
                           close_order_series, title, n_points_visible=500_000)

In [23]:
widget.show()